# Constructing our Minimum-Covariance Portfolios

We will be constructing our portfolios by selecting the assets which have the lowest correlations between each other. We will rebalance semi-annually based on these selections, and at each rebalance, ensure that our portfolio is split among the two geographies 50/50 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Remove 1st line since they are pct changes, therefore have
# NaNs in the first row
rets = pd.read_csv("returns.csv")[1:]
rets.head()

FileNotFoundError: [Errno 2] No such file or directory: 'returns.csv'

In [ ]:
# Reindex to date
rets.index = pd.to_datetime(rets['Date'])
del rets['Date']
rets.head()

In [ ]:
rets.index

In [ ]:
# Create a list of semiannual dates to split on
dates_to_split = pd.date_range(rets.index[0], rets.index[-1], freq='6M')
dates_to_split

In [ ]:
# Split on these dates
semiannual = {}

for i in range(len(dates_to_split)-1):
    #print(dates_to_split[i+1])
    semiannual[i] = rets[dates_to_split[i]:dates_to_split[i+1]]

In [ ]:
# Sanity check
print(semiannual[0].index[0],semiannual[0].index[-1])

In [ ]:
# Sanity check
print(semiannual[1].index[0],semiannual[1].index[-1])

In [ ]:
# Sanity check
print(semiannual[19].index[0],semiannual[19].index[-1])

In [ ]:
# For each group of 6mo, find the assets with lowest avg correlation
corrs = semiannual[10].corr().abs().mean(axis=1)
corrs

In [ ]:
# Sort those values to see which ones have lowest correlation
corrs.sort_values(axis=0)[:30]

In [ ]:
# First 15 CAN values
[x for x in corrs.sort_values(axis=0).index if "CAN_" in x][:15]

In [ ]:
# Loop through each semiannual period, save the 15 Canadian and USA assets with the
# lowest average correlation
top15 = {}
for i in range(1,len(semiannual)):
    corrs = semiannual[i-1].corr().abs().mean(axis=1)
    
    top15[i] = [x for x in corrs.sort_values(axis=0).index if "USA_" in x][:15]
    top15[i] += [x for x in corrs.sort_values(axis=0).index if "CAN_" in x][:15]
    print(i)

In [ ]:
# Sanity check
top15[1]

In [ ]:
# We only need the index
top15[1]

In [ ]:
# Let's create a one-period example before looping through them all, start
# with 100k capital
capital = 100000

In [ ]:
# Use previous period's lowest corrs to construct portfolio
# for current period
returns = semiannual[1][top15[1]]
returns

In [ ]:
# Save the sum of the dollar returns, add to capital
capital += returns.cumsum().iloc[-1,:].sum()
capital

In [ ]:
# Sanity check to make sure calculation is correct
returns['EPU'].cumsum().plot()

In [ ]:
# Reallocate capital and rebalance
position_size = capital / 30
position_size

In [ ]:
# Use new allocations to calculate next period
returns2 = semiannual[2][top15[2]] * position_size
returns2

In [ ]:
# Concatenate to get continuous stream
pd.concat([returns, returns2])

In [ ]:
# Generalize to all time periods

capital = 100000
dollar_full_portfolio = pd.DataFrame()
pct_full_portfolio = pd.DataFrame()
PnL = {}

for i in range(1,len(semiannual.keys())):
    position_size = capital / 30
    returns = semiannual[i][top15[i]]
    
    
    # Remove an outlier
    returns[returns.values > 100] = 0
    
    pct_full_portfolio = pd.concat([pct_full_portfolio, returns])
    dollar_full_portfolio = pd.concat([dollar_full_portfolio, returns*position_size])
    
    capital += sum(returns.iloc[-1,:])
    
    PnL[i] = returns.sum(axis=1)

In [ ]:
dollar_full_portfolio = dollar_full_portfolio.fillna(0)

In [ ]:
dollar_full_portfolio.cumsum().plot(figsize = (15,8))

In [ ]:
# PnL of whole portfolio 
(pd.concat(PnL)* 100000).cumsum().plot(figsize=(15,8))

In [ ]:
PnL[1]

In [ ]:
import importlib
import Risk_analytics
importlib.reload(Risk_analytics)
from Risk_analytics import risk
returns 


In [ ]:
pd.DataFrame(np.cov(returns))

In [ ]:
import importlib
import Risk_analytics
importlib.reload(Risk_analytics)
from Risk_analytics import risk
risk_profile = risk(returns,np.ones(15),15)
risk_profile

In [ ]:
risk_profile.plot_VaR(1000)

In [ ]:
risk_profile.max_drawdown()

In [ ]:
risk_profile.sharpe_ratio(0.02)